# Sarcasm Detection Demo

This notebook illustrates how to use a **pre-trained AI model** to detect sarcasm in text. 
We will not train a model from scratch. Instead, we use a model that has already learned from millions of tweets.

**Steps:**
1. Load the pre-trained model.
2. Run the model on example sentences.
3. See how well it performs (Accuracy & Confusion Matrix).

In [ ]:
# 1. Setup and Imports
# We import the necessary libraries.
# 'transformers' allows us to use the pre-trained model.
# 'pandas' helps us manage data.
# 'seaborn' and 'matplotlib' are for plotting the graph.

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from scipy.special import softmax
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

# This setup ensures graphs look nice
sns.set_theme(style="whitegrid")

In [ ]:
# 2. Load the Pre-trained Model
# We are using a RoBERTa model trained specifically on tweets to detect irony/sarcasm.
# Model Source: cardiffnlp/twitter-roberta-base-irony

MODEL_NAME = "cardiffnlp/twitter-roberta-base-irony"

print("Loading model... (this may take a moment)")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
print("Model loaded successfully!")

# Function to simplify prediction
def predict_sarcasm_score(text):
    # 1. Prepare text for the model
    encoded_input = tokenizer(text, return_tensors='pt')
    
    # 2. Get the model's raw output
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    
    # 3. Convert scores to probabilities (0 to 1)
    scores = softmax(scores)
    
    # The model returns [Probability of NOT Sarcastic, Probability of Sarcastic]
    # We return the label with the highest probability
    # 1 = Sarcastic (irony), 0 = Not Sarcastic
    prediction_index = np.argmax(scores)
    
    label = "Sarcastic" if prediction_index == 1 else "Literal"
    confidence = scores[prediction_index]
    return label, confidence

In [ ]:
# 3. Run Inference on Examples
# Here is a small list of examples to test the model (Gold Standard Data).

examples = [
    # Literal Statements (Should be 0/Literal)
    {"text": "I love my job, it is very rewarding.", "truth": "Literal"},
    {"text": "The sky is blue today.", "truth": "Literal"},
    {"text": "I missed the bus and now I am late.", "truth": "Literal"},
    
    # Sarcastic Statements (Should be 1/Sarcastic)
    {"text": "Oh great, I missed the bus again. Just what I needed.", "truth": "Sarcastic"},
    {"text": "I love working on weekends.", "truth": "Sarcastic"},
    {"text": "Thanks for the help, you were completely useless.", "truth": "Sarcastic"}
]

# We process each example
results = []
print(f"{'Text':<50} | {'Prediction':<10} | {'Truth':<10}")
print("-"*80)

for item in examples:
    text = item["text"]
    truth = item["truth"]
    
    # Ask the AI
    prediction, conf = predict_sarcasm_score(text)
    
    # Save result
    results.append({"text": text, "truth": truth, "prediction": prediction})
    
    print(f"{text[:47]+'...':<50} | {prediction:<10} | {truth:<10}")

In [ ]:
# 4. Evaluation Metrics
# Let's see the overall accuracy and where mistakes happened.

df_results = pd.DataFrame(results)

# Compute Accuracy
acc = accuracy_score(df_results['truth'], df_results['prediction'])
print(f"\nOverall Model Accuracy: {acc*100:.2f}%")

# Create Confusion Matrix
# This shows: How many Literal were correctly detected? How many Sarcastic were missed?
labels = ["Literal", "Sarcastic"]
cm = confusion_matrix(df_results['truth'], df_results['prediction'], labels=labels)

# Plotting
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix: Human vs AI')
plt.show()